In [3]:
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

In [5]:
# begin code
url_links = []
property_metadata = defaultdict(dict)
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

url= "https://www.domain.com.au/rent/melbourne-region-vic/?sort=price-desc&page=1"
bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")
bs_object.find("h1", {"class": "css-164r41r"}).text

AttributeError: 'NoneType' object has no attribute 'text'